## N-Body GPU-accelerated Simulation in Python using Numba and CuPy
*By Adrien GAGNE*

### Introduction

>This project is a proof of concept, the goal is to render the formation of galaxy clusters with python. It is a way for me to learn and manipulate complex 3D simulations. This notebook is the result of multiple weeks of trials and errors that leaded to a functional simulation.

>I chose the format of the notebook so that I could document my journey, but also for people to modify it and upgrade it. You are thus most welcome to send me your modifications on this project !

>To compute a N-Body simulation I initially intended to use a simple "every body interacts with each other" method, but the time complexity of such algorithm is $O(N^2)$, which is not ideal. I then setteled on a classic Perodic Barnes Hut method $O(Nlog(N))$ with a stack-based traversal on the GPU, that will grant me the ability to render **1 million** bodies!

>To meet the goal of a large-scale structure formation simulation, the intial cosmological conditions will use the Zel'dovich approximation.

#### Project Roadmap

**Phase 1: Configuration & Cosmological Initial Conditions**
- **1.1 Global Configuration**: Definition of physical constants ($G$, $H_0$), simulation parameters (N, box size, $\theta$ opening angle), and CUDA block configurations.

- **1.2 Zel'dovich Approximation (Initial Conditions)**: Implementation of a Gaussian Random Field generator using FFT (via CuPy) to displace particles from a uniform grid. This creates the initial density perturbations required for structure formation.

**Phase 2: GPU Linear Octree Construction**
- **2.1 Morton Encoding (Z-Order Curve)**: Kernel to compute Morton codes for all particles to map 3D positions to 1D indices.

- **2.2 Radix Sort**: Sorting particles on the GPU based on Morton codes to ensure spatial locality.

- **2.3 Tree Building (Linear Octree)**: A bottom-up approach to construct the tree hierarchy (leaf nodes $\to$ internal nodes) directly in GPU memory, avoiding pointer-chasing structures.

- **2.4 Multipole Moments**: Kernel to compute total mass and Center of Mass (COM) for each node in the tree (upward pass).

**Phase 3: Physics Engine (The Kernel)**
- **3.1 Stack-Based Tree Traversal**: The core logic. A CUDA kernel using local thread memory (simulated stack) to traverse the Octree without recursion.

- **3.2 Force Calculation**: Computing gravitational acceleration with the Barnes-Hut acceptance criterion (MAC).

- **3.3 Hubble Drag & Integration**: Applying the expanding universe drag term and updating particle kinematics using a Symplectic integrator.

**Phase 4: In-Situ GPU Visualization**
- **4.1 Density Projection Kernel**: A "Splatting" kernel that projects 3D particles onto a 2D accumulation buffer directly in GPU memory.

- **4.2 Log-Scale Transfer Function**: Converting density values to RGB colors (mimicking the video's heat-map aesthetic) without CPU transfer.

- **4.3 Output Pipeline**: Displaying the GPU buffer in the Jupyter environment.

**Phase 5: Orchestration & Validation**
- **5.1 Verification Cell**: Line-by-line testing logic.

- **5.2 Simulation Loop**: The main driver loop handling time-stepping and frame rendering.

#### Requirements

>This project was written using a Geforce RTX 5070, it may not work (Performance wise) the same way on other devices. This project uses the following libraries

- Numba
- CuPy
- Numba
- Vispy
- TQDM

### Phase 1: Configuration & Cosmological Initial Conditions

#### 1.1 Global Configuration

In [ ]:
import numpy as np
import cupy as cp
from cupy.fft import fftn, ifftn
import numba
from numba import cuda, uint64
from PIL import Image
from IPython.display import display, clear_output
import math
import os
import subprocess
from tqdm import tqdm
import concurrent.futures
import time

>For physics calculations units are arbitrary (called code units)

In [ ]:
# --- SIMULATION PARAMETERS ---
N_GRID = 100                  # Grid size per dimension
N_PARTICLES = N_GRID**3      # Total particles (~262k for 64^3, 1M for 100^3)
BOX_SIZE = 100.0             # Length of the periodic box side
SOFTENING = 0.1              # Softening length to prevent singularities at r=0
G = 1.0                      # Gravitational constant (code units)
H0 = 50.0                     # Hubble parameter (expansion rate)
DT = 0.005                    # Time step
N_STEPS = 414                   # Total simulation steps
A_START = 0.01               # Starting scale factor
A_END = 1.0                 # Ending scale factor
OMEGA_M = 1.0              # Matter density parameter

In [ ]:
# --- BARNES-HUT PARAMETERS ---
THETA = 0.5                  # Opening angle for MAC (0.5 is standard trade-off)
MAX_DEPTH = 18               # Max tree depth (octree)
WARP_SIZE = 32

In [ ]:
# --- VISUALIZATION PARAMETERS ---
RES = 1024                   # Render resolution (RES x RES)
CMAP_GAMMA = 0.5             # Gamma correction for log-scale brightness
OUTPUT_DIR = "render_output"
VIDEO_NAME = "nbody_simulation.mp4"

FOV = 60.0
CAM_DIST_MULT = 1.8
ROTATION_SPEED = 0.005 #Radians per frame
os.makedirs(OUTPUT_DIR, exist_ok=True) 

In [ ]:
# --- CUDA CONFIGURATION ---
TPB = 256                    # Threads Per Block
BPG = (N_PARTICLES + TPB - 1) // TPB # Blocks Per Grid
BATCH_SIZE = 16384
BATCH_BPG = (BATCH_SIZE + TPB - 1) // TPB
MAX_H_RATE = 1000.0 

In [ ]:
# --- TREE CONSTRUCTION ---
num_nodes = 2 * N_PARTICLES

In [ ]:
# --- PHYSICS COMPUTATIONS ---
drag_factor = 1.0 / (1.0 + H0 * DT)

#### 1.2 : Zel'Dovich Approximation

>Intial testing showed that a random repartitions of bodies in space will ultimately result in a global collapse in the center of the box or an equilibrium of all the bodies. No cluster appeared (renders without initial conditions)

![Test Render](resources/1_1_test_animation.webp "Example 1")

![Test Render 2](resources/1_2_test_render.webp "Example 2")

>It makes for a nice "supernova" style simulation, but that wasn't planned at all!

>The [Zeldovich approximation](https://en.wikipedia.org/wiki/Zeldovich_approximation) is used here to introduce a nonlinear evolution for the structure. The mathematical aspect of this is to use the following mass density:
>$$\rho(\mathbf{x}, t) = \frac{\rho_0}{\det \left( \frac{\partial \mathbf{x}}{\partial \mathbf{q}} \right)} = \frac{\rho_0}{\det \left( \mathbf{I} + \sigma(t) \frac{\partial \mathbf{u}}{\partial \mathbf{q}} \right)}$$
>where:
>- $\rho_0$ is the mean background density,
>- $x(t)$ is the comoving position (position that does not take into account space expanse) of a particle,
>- $\sigma(t)$ is the linear growth factor of density perturbations,
>- $u$ is the initial displacement vector determined by the initial density field
>- $q$ is the initial lagrangian position of the particle

>Implementing this is done by generating a Gaussian random field inside our box and applying a power spectrum function $P(k) = 1/k^3$ and then compute the displacement in the Fourier space.

In [ ]:
def generate_zeldovich_ics(n_grid, box_size, seed=42, spectral_index=2.0, amplitude=1.0):
    """
    Generates Initial Conditions using the Zel'dovich approximation.
    Method:
    1- Generate a uniform 3D grid of particles (Lagrangian coordinates q).
    2- Generate a Gaussian Random Field in Fourier space.
    3- Apply a power spectrum P(k) ~ k^-n.
    4- Compute displacement field via FFT.
    5- Displace particles and assign initial velocities (Hubble flow + peculiar velocity).
    """
    cp.random.seed(seed)

    lins = cp.linspace(0, box_size, n_grid, endpoint=False) # Lagrangian grid

    qx, qy, qz = cp.meshgrid(lins, lins, lins, indexing='ij') # coordinates of q (see above)
    
    # Fourier Space Setup
    k = cp.fft.fftfreq(n_grid, d=box_size/n_grid) * 2 * cp.pi
    kx, ky, kz = cp.meshgrid(k, k, k, indexing='ij')
    k_sq = kx**2 + ky**2 + kz**2 # coordinates in fourier space (squared)
    k_sq[0,0,0] = 1e-10 # Avoid division by zero at DC component

    # The random Gaussian field in Fourier space
    random_field = cp.random.normal(0, 1, (n_grid, n_grid, n_grid)) + \
                   1j * cp.random.normal(0, 1, (n_grid, n_grid, n_grid))
    

    # aplly power spectrum
    power_spectrum = 1.0 / (k_sq ** (spectral_index / 2.0)) # P(k) ~ k^-2
    power_spectrum[0,0,0] = 0
    delta_k = random_field * cp.sqrt(power_spectrum)

    #Compute displacement field
    def get_displacement(k_component_grid):
        term = 1j * k_component_grid / k_sq
        term[0,0,0] = 0
        disp_k = term * delta_k
        return cp.fft.ifftn(disp_k).real * n_grid**3 
    
    dx = get_displacement(kx).flatten()
    dy = get_displacement(ky).flatten()
    dz = get_displacement(kz).flatten()

    dx *= amplitude
    dy *= amplitude
    dz *= amplitude

    
    # x = q + D * dx with perodic wrap
    pos_x = cp.mod(qx.flatten() + dx, box_size).astype(cp.float32)
    pos_y = cp.mod(qy.flatten() + dy, box_size).astype(cp.float32)
    pos_z = cp.mod(qz.flatten() + dz, box_size).astype(cp.float32)

    H_start = H0 * (A_START ** (-1.5))
    vel_factor = H_start * A_START * 0.8 #0.8 is arbitrary growth rate

    #0.5 is arbitrary growth rate
    vel_x = (dx * vel_factor * 0.5).astype(cp.float32)
    vel_y = (dy * vel_factor * 0.5).astype(cp.float32)
    vel_z = (dz * vel_factor * 0.5).astype(cp.float32)

    mass = cp.ones(n_grid**3, dtype=cp.float32) # Uniform mass
    
    return (pos_x, pos_y, pos_z), (vel_x, vel_y, vel_z), mass

### Phase 2 : Linear Octree Construction

>The Barnes-Hut algorithm makes approximations, It considers that if $N_1$ bodies are close enough together they can be computed as a single mass body $m = \sum_{i}^{N_1}(m_i)$.\
>To represent the 3-dimensional space, it uses an **Octree** (tree with strictly 8 branchs per node), the resolution per octant is determined by the amount of bodies in the octant. The search complexity for such tree is $O(LogN)$.

#### 2.1 : Morton Encoding

>The only issue here: GPU's don't like trees, or multiple dimension objects in general, what they really like is arrays (vectors). This is why everything (coordinates, octree, etc...) has to be "flattened" into arrays.\
>For the coordinates of the positions, velocities and accelerations, I found the Z-curve encoding to be good enough performance wise. Also called the **Morton encoding** it consist in interleaving coordinates together to obtain a single number.

>Let's take the binary representation of the coordinates of the position of a body inside our space:
>- $X = x_1x_2x_3$
>- $Y = y_1y_2y_3$
>- $Z = z_1z_2z_3$

>The Morton encoding interleaves those bits together in the following way:
>- $CODE = z_3y_3x_3z_2y_2x_2z_1y_1x_1$

In [ ]:
@cuda.jit(device=True)
def expand_bits(v):
    """
    Expands a 21-bit integer into 64 bits by inserting 2 zeros after each bit.
    """
    v &= 0x1fffff # avoid overflow
    v = (v | (v << 32)) & 0x1f00000000ffff
    v = (v | (v << 16)) & 0x1f0000ff0000ff
    v = (v | (v << 8))  & 0x100f00f00f00f00f
    v = (v | (v << 4))  & 0x10c30c30c30c30c3
    v = (v | (v << 2))  & 0x1249249249249249
    return v

In [ ]:
@cuda.jit(device=True)
def get_morton_code(x, y, z, min_grid, grid_scale):
    """
    Computes the 3D Morton code for a body given its position.
    """

    ix = uint64((x - min_grid) * grid_scale)
    iy = uint64((y - min_grid) * grid_scale)
    iz = uint64((z - min_grid) * grid_scale)

    return expand_bits(ix) | (expand_bits(iy) << 1) | (expand_bits(iz) << 2)

In [ ]:
@cuda.jit
def compute_morton_codes(pos_x, pos_y, pos_z, codes, n_bodies, grid_scale, indices):
    """
    CUDA kernel to compute Morton codes for all particles.
    """
    idx = cuda.grid(1)
    if idx < n_bodies:
        codes[idx] = get_morton_code(pos_x[idx], pos_y[idx], pos_z[idx], 0.0, grid_scale)
        indices[idx] = idx # keep track of original indices

#### 2.2 : Radix Sort

>The final array `codes` is big, an important step is to optimize the search inside of it by sorting it. Since the programm will mostly seek for neighbors when processing a body, It needs to be sorted relative to "body proximity".
>To explain it better, sorting the morton codes will improve memory locality (Coalescing).

In [ ]:
def sort_bodies(pos, vel, mass):
    """
    Radix sort all arrays based on their Morton codes.
    """
    n = pos[0].shape[0]
    min_coord = 0.0
    grid_scale = ((2**21 - 2)/ BOX_SIZE) #Boundary issue solved by -2

    codes = cp.zeros(n, dtype=cp.uint64)
    indices = cp.zeros(n, dtype=cp.uint32)

    compute_morton_codes[BPG, TPB](pos[0], pos[1], pos[2], codes, n, grid_scale, indices)

    sort_idx = cp.argsort(codes) #Radix sort on GPU

    pos_sorted = (pos[0][sort_idx], pos[1][sort_idx], pos[2][sort_idx])
    vel_sorted = (vel[0][sort_idx], vel[1][sort_idx], vel[2][sort_idx])
    mass_sorted = mass[sort_idx]
    codes_sorted = codes[sort_idx]

    return pos_sorted, vel_sorted, mass_sorted, codes_sorted

#### 2.3 : Tree Building

>Now that was the easy part. The tree construction is going to be a bit more complex.\
>Fortunately, Nvidia researcher [**Tero Karras**](https://research.nvidia.com/person/tero-karras) published a really useful paper on [tree construction optimized for parallelism](https://research.nvidia.com/sites/default/files/pubs/2012-06_Maximizing-Parallelism-in/karras2012hpg_paper.pdf). The concept is rather simple:

> - **The Linear Part**: Since GPU's work best with an 1D array and the tree is an *Octree*, then to get from node $i$ to it's fist child node, the math is the following $i_{child} = 8*i + 1$
>- **The Parallelism Part**: Since the list of coordinates is already sorted (with *Morton Encoding*) the tree structure can be figured out just by looking at the bits of the Morton code of neighboring points.\
>If two adjacent points in the list share the first 3 bits of their code, they belong in the same "large" box. If they share the first 6 bits, they belong in the same "medium" sub-box.\
The point where the bits differ is exactly where the tree splits

In [ ]:
@cuda.jit(device=True)
def delta_fn(codes, i, j, n):
    """
    Computes the length of the longest common prefix between codes[i] and codes[j].
    Handles boundary conditions.
    """
    if j < 0 or j >= n:
        return -1
    
    code_i = codes[i]
    code_j = codes[j]

    if code_i == code_j :
        code_i, code_j = i, j # if duplicate codes
        return 64 + (32 - cuda.clz(i ^ j)) #32 for int32 index collision
    
    return cuda.libdevice.clzll(numba.int64(code_i ^ code_j)) # CLZ shows shared prefix and XOR differing bits (czll because int64)

In [ ]:
@cuda.jit
def build_radix_tree_kernel(codes, children, parents):
    """
    Constructs the internal nodes of the binary radix tree.
    Each thread i (0 to N-2) constructs internal node (i + N).
    """

    i = cuda.grid(1)
    n = codes.shape[0]

    if i >= n-1:
        return
    
    idx = i + n #internal node index

    d_prev = delta_fn(codes, i, i-1, n)
    d_next = delta_fn(codes, i, i+1, n)

    direction = 1
    min_delta = d_prev
    if d_next > d_prev:
        direction = 1
        min_delta = d_prev # Lower bound for split
    else:
        direction = -1
        min_delta = d_next
        
    
    l_max = 2
    while delta_fn(codes, i, i + l_max * direction, n) > min_delta: # Determine upper bound of the range (l_max)
        l_max *= 2 
        
    
    l = 0
    t = l_max // 2
    while t >= 1:
        if delta_fn(codes, i, i + (l + t) * direction, n) > min_delta: #Find the other end using binary search
            l += t
        t //= 2
        
    j = i + l * direction
    
    
    delta_node = delta_fn(codes, i, j, n) 
    s = 0
    t = l
    while t > 0: # Find the split position (gamma)
        
        step = (t + 1) // 2 

        check_idx = i + (s + step) * direction
        if delta_fn(codes, i, check_idx, n) > delta_node: # Standard binary search to find split
            s += step
        t -= step
        
    gamma = i + s * direction
    if direction == -1:
        gamma -= 1 # Adjustment for reverse search

    # Left child
    left = gamma
    right = gamma + 1
    
    range_left = min(i, j)
    range_right = max(i, j)
    

    if range_left == gamma: #assign Left
        children[idx, 0] = gamma #leaf
    else:
        children[idx, 0] = gamma + n #internal Node
        

    if range_right == gamma + 1: #assign Right
        children[idx, 1] = gamma + 1 # Leaf
    else:
        children[idx, 1] = gamma + 1 + n # Internal Node

    
    parents[children[idx, 0]] = idx # Set Parents (needed for upward pass)
    parents[children[idx, 1]] = idx

#### 2.4 : Multipole Moments

>Now, every "Bodies" have coordinates, initial velocities and a way to efficiently map them in space. But to compute the interactions in the Barnes-Hut method, every nodes need to have a mass value.\
>For every leafs the mass is already known and the center of mass (CoM) is their positions. The sum of those mass makes for the mass of the nodes and the CoM can be computed the same way.\
>To make those calculations easier to implement, and because the tree is easily traversed both way, the mass can be propagated from the leaf to the root.

In [ ]:
@cuda.jit
def compute_multipoles_kernel(pos, mass, children, parents, node_mass, node_com, 
                              node_min, node_max, counters):
    """
    Computes Mass, Center of Mass, and Bounding Box (AABB) for each node.
    """
    idx = cuda.grid(1)
    n_leaf = pos[0].shape[0]

    pos_x, pos_y, pos_z = pos

    if idx >= n_leaf:
        return

    #Initialize Leaf Nodes
    m = mass[idx]
    px = pos_x[idx]
    py = pos_y[idx]
    pz = pos_z[idx]

    node_mass[idx] = m
    node_com[idx, 0] = px
    node_com[idx, 1] = py
    node_com[idx, 2] = pz
    
    # Initialize AABB (Leaf min = max = pos)
    node_min[idx, 0] = px
    node_min[idx, 1] = py
    node_min[idx, 2] = pz
    node_max[idx, 0] = px
    node_max[idx, 1] = py
    node_max[idx, 2] = pz
    
    curr = idx

    while True:
        parent = parents[curr]
        if parent == -1:
            break 
        
        cuda.threadfence()
        old_val = cuda.atomic.add(counters, parent, 1)

        if old_val == 0: 
            break # Wait for sibling

        elif old_val == 1: # Second child creates the parent
            left = children[parent, 0]
            right = children[parent, 1]
            
            #mass and COM 
            m_l = node_mass[left]
            m_r = node_mass[right]
            sum_m = m_l + m_r
            
            if sum_m > 0:
                inv_m = 1.0 / sum_m
                cx = (node_com[left, 0] * m_l + node_com[right, 0] * m_r) * inv_m
                cy = (node_com[left, 1] * m_l + node_com[right, 1] * m_r) * inv_m
                cz = (node_com[left, 2] * m_l + node_com[right, 2] * m_r) * inv_m
            else:
                cx, cy, cz = 0.0, 0.0, 0.0

            node_mass[parent] = sum_m
            node_com[parent, 0] = cx
            node_com[parent, 1] = cy
            node_com[parent, 2] = cz
            
            # AABB merge
            
            # Min X
            lx, rx = node_min[left, 0], node_min[right, 0]
            node_min[parent, 0] = lx if lx < rx else rx
            # Min Y
            ly, ry = node_min[left, 1], node_min[right, 1]
            node_min[parent, 1] = ly if ly < ry else ry
            # Min Z
            lz, rz = node_min[left, 2], node_min[right, 2]
            node_min[parent, 2] = lz if lz < rz else rz
            
            # Max X
            lx, rx = node_max[left, 0], node_max[right, 0]
            node_max[parent, 0] = lx if lx > rx else rx
            # Max Y
            ly, ry = node_max[left, 1], node_max[right, 1]
            node_max[parent, 1] = ly if ly > ry else ry
            # Max Z
            lz, rz = node_max[left, 2], node_max[right, 2]
            node_max[parent, 2] = lz if lz > rz else rz

            curr = parent

>Up to this point a "Sanity check" was important, in an ideal world the sum of the masses of the **leaves** had to be the sum of the masses of the **nodes**. But when comparing thoses values after multiple computations, a mass mismatch of ~$3\%$ was observed.\
>Proably due to race conditions in memory, the `cuda.threadfence()` fixes it.

### Phase 3: Physics Engine

>This part is the heart of this project, the challenge is to calculate the gravitational force between every nodes and leaves. wheras the standard Barnes-Hut is recursive, this one cannot. The solution is to make it stack-less transversal:
>- Simulate a stack using fixed-size array
>- Push the root node onto the stack
>- while the stack is not empty :
>    - pop a node
>    - check criteria ($disatnce/size > \theta$)
>    - if it is `True` compute force using node's mass
>    - else, it's a leaf compute direct force

>Regarding periodic boundaries, for a cosmologically accurate simulation, the formula for the forces at stake near the edge of the box is the following:
>$$\mathbf{f}_{real} = - \frac{G M}{r^3} \mathbf{r} \left[ \text{erfc}(\alpha r) + \frac{2\alpha r}{\sqrt{\pi}} \exp(-\alpha^2 r^2) \right]$$
>This is called the **Ewald Summation**, $\alpha$ being the term controlling the split between the FFT and the Tree.

>However, as this project is a visual proof of concept, it will use the **Nearest Image approximation** for the box edges. When calculating the distance between particle $i$ and node $j$, we choose the periodic image of $j$ closest to $i$.
>- `dx = x_j - x_i`
>- `dx = dx - box_size * round(dx / box_size)`

In [ ]:
@cuda.jit
def find_root_kernel(parents, root_idx):
    i = cuda.grid(1)
    if i < parents.shape[0]:
        if parents[i] == -1:
            root_idx[0] = i

In [ ]:
@cuda.jit(fastmath=True)
def compute_forces_kernel(pos, mass, children, node_mass, node_com, 
                          node_min, node_max, force,
                          theta, G, softening, box_size, root_idx, i_offset):
    tid = cuda.grid(1)

    i = tid + i_offset

    if i >= pos[0].shape[0]:
        return

    p_pos_x = pos[0][i]
    p_pos_y = pos[1][i]
    p_pos_z = pos[2][i]

    acc_x = 0.0
    acc_y = 0.0
    acc_z = 0.0

    # Stack initialization
    stack = cuda.local.array(96 , dtype=numba.int32) 
    stack_top = 0
    stack[stack_top] = root_idx[0]
    stack_top += 1
    
    theta_sq = theta * theta

    while stack_top > 0:
        stack_top -= 1
        node_idx = stack[stack_top]
        
        # Load Mass
        n_mass = node_mass[node_idx]
        
        #Skip empty nodes or extremely light nodes
        if n_mass <= 0:
            continue
            
        # Distance Calculation (Nearest Image)
        dx = node_com[node_idx, 0] - p_pos_x
        dy = node_com[node_idx, 1] - p_pos_y
        dz = node_com[node_idx, 2] - p_pos_z
        
        dx -= box_size * round(dx / box_size)
        dy -= box_size * round(dy / box_size)
        dz -= box_size * round(dz / box_size)
        
        dist_sq = dx*dx + dy*dy + dz*dz + 1e-10
        
        is_leaf = (children[node_idx, 0] == -1)
        apply_force = is_leaf
        
        if not is_leaf:
            # AABB Size Calculation
            sx = node_max[node_idx, 0] - node_min[node_idx, 0]
            sy = node_max[node_idx, 1] - node_min[node_idx, 1]
            sz = node_max[node_idx, 2] - node_min[node_idx, 2]
            
            # Clamp size for periodic boundary spanning nodes
            # If a node spans > 50% of the box, it wraps.
            if sx > box_size * 0.5: sx = box_size
            if sy > box_size * 0.5: sy = box_size
            if sz > box_size * 0.5: sz = box_size
            
            size_sq = sx*sx + sy*sy + sz*sz # Diagonal squared usually safer
            
            # MAC Criterion
            if size_sq < (theta_sq * dist_sq):
                apply_force = True
        
        # 2. Avoid Self-Interaction
        if node_idx == i:
            apply_force = False

        if apply_force:
            dist_soft = dist_sq + softening*softening
            inv_dist = 1.0 / math.sqrt(dist_soft)
            inv_dist_cube = inv_dist * inv_dist * inv_dist
            f = G * n_mass * inv_dist_cube
            
            acc_x += f * dx
            acc_y += f * dy
            acc_z += f * dz
            
        elif not is_leaf: 
            # Stack Safety: Only push if there is room
            if stack_top + 2 < 96:
                stack[stack_top] = children[node_idx, 0]
                stack_top += 1
                stack[stack_top] = children[node_idx, 1]
                stack_top += 1

    force[i, 0] = acc_x
    force[i, 1] = acc_y
    force[i, 2] = acc_z

In [ ]:
@cuda.jit
def integrate_kernel(pos, vel, force, dt, box_size, drag_factor):
    """
    Updates positions and velocities.
    Includes 'drag_factor' to simulate Hubble expansion drag (a(t)).
    v_{i+1} = v_i + a * dt
    x_{i+1} = x_i + v_{i+1} * dt
    """
    i = cuda.grid(1)
    if i >= pos[0].shape[0]:
        return

    pos_x, pos_y, pos_z = pos
    vel_x, vel_y, vel_z = vel

    vx = vel_x[i] + force[i, 0] * dt
    vy = vel_y[i] + force[i, 1] * dt
    vz = vel_z[i] + force[i, 2] * dt

    vx *= drag_factor
    vy *= drag_factor
    vz *= drag_factor
    
    max_v = box_size * 0.1 / dt
    v_sq = vx*vx + vy*vy + vz*vz
    if v_sq > max_v*max_v:
        scale = max_v / math.sqrt(v_sq)
        vx *= scale
        vy *= scale
        vz *= scale

    # Update Position
    px = pos_x[i] + vx * dt
    py = pos_y[i] + vy * dt
    pz = pos_z[i] + vz * dt

    # Periodic Wrap
    px = px % box_size
    py = py % box_size
    pz = pz % box_size
    
    if px < 0: px += box_size
    if py < 0: py += box_size
    if pz < 0: pz += box_size

    # Store
    vel_x[i] = vx
    vel_y[i] = vy
    vel_z[i] = vz
    pos_x[i] = px
    pos_y[i] = py
    pos_z[i] = pz

https://people.eecs.berkeley.edu/~kubitron/courses/cs262a-F21/projects/reports/project16_report_ver3.pdf

### Phase 4 : Visuals

>The slow part of rendering is the data traveling back and forth from the GPU to the CPU. Fixing that is doing point splatting instead of rendering gaussian spheres.\
>For the resolution of $1024^2$, the $10^6$ bodies will create perfect aliased density map. Then applying a logarithmic transfer function will make the filaments appear while preventing big clusters to shine too bright.

>The following functions run on CPU as they only run once per frame

In [ ]:
def normalize(v):
    norm = np.linalg.norm(v)
    return v if norm == 0 else v / norm

def look_at(eye, target, up):
    """
    Creates a View Matrix.
    """
    z = normalize(eye - target)
    x = normalize(np.cross(up, z))
    y = normalize(np.cross(z, x))

    view = np.identity(4, dtype=np.float32)
    view[0, :3] = x
    view[1, :3] = y
    view[2, :3] = z
    view[0, 3] = -np.dot(x, eye)
    view[1, 3] = -np.dot(y, eye)
    view[2, 3] = -np.dot(z, eye)
    
    return view

def perspective(fov_deg, aspect, near, far):
    """
    Creates a Perspective Projection Matrix.
    """
    fov_rad = np.radians(fov_deg)
    f = 1.0 / np.tan(fov_rad / 2.0)
    
    proj = np.zeros((4, 4), dtype=np.float32)
    proj[0, 0] = f / aspect
    proj[1, 1] = f
    proj[2, 2] = (far + near) / (near - far)
    proj[2, 3] = (2 * far * near) / (near - far)
    proj[3, 2] = -1.0
    
    return proj

def get_mvp_matrix(step, box_center, box_size):
    """
    Calculates the full MVP matrix for the current time step.
    Orbit logic: Circular path on XZ plane, looking at center.
    """
    angle = step * ROTATION_SPEED
    radius = box_size * CAM_DIST_MULT
    
    # Orbiting position
    eye_x = box_center[0] + radius * np.cos(angle)
    eye_y = box_center[1] + (box_size * 0.2) # Slight elevation
    eye_z = box_center[2] + radius * np.sin(angle)
    
    eye = np.array([eye_x, eye_y, eye_z], dtype=np.float32)
    target = np.array(box_center, dtype=np.float32)
    up = np.array([0, 1, 0], dtype=np.float32)
    
    view = look_at(eye, target, up)
    proj = perspective(FOV, 1.0, 0.1, radius * 3.0)
    
    # Model matrix is Identity (simulation coordinates are world coordinates)
    mvp = np.dot(proj, view)
    return mvp

In [ ]:
@cuda.jit
def render_3d_density_kernel(pos, grid, mvp, width, height):
    """
    Projects 3D world coordinates to 2D screen coordinates using MVP matrix.
    Performs perspective division and atomic accumulation.
    """
    i = cuda.grid(1)
    if i < pos[0].shape[0]:
        # Load Particle Position (x, y, z, 1.0)
        px = pos[0][i]
        py = pos[1][i]
        pz = pos[2][i]
        pw = 1.0
        
        # Matrix Multiplication: v_clip = MVP * v_world
        # Rows of MVP are hardcoded for performance (unroll loop)
        c_x = mvp[0,0]*px + mvp[0,1]*py + mvp[0,2]*pz + mvp[0,3]*pw
        c_y = mvp[1,0]*px + mvp[1,1]*py + mvp[1,2]*pz + mvp[1,3]*pw
        c_z = mvp[2,0]*px + mvp[2,1]*py + mvp[2,2]*pz + mvp[2,3]*pw
        c_w = mvp[3,0]*px + mvp[3,1]*py + mvp[3,2]*pz + mvp[3,3]*pw
        
        # Frustum Culling (simple w check)
        if c_w <= 0.001:
            return
            
        # Perspective Divide (Clip Space -> NDC)
        inv_w = 1.0 / c_w
        ndc_x = c_x * inv_w
        ndc_y = c_y * inv_w
        ndc_z = c_z * inv_w # Depth, not used for splatting but good to have
        
        # Viewport Transform (NDC -> Screen Coordinates)
        # NDC is [-1, 1]. Map to [0, width]
        screen_x = (ndc_x + 1.0) * 0.5 * width
        screen_y = (1.0 - ndc_y) * 0.5 * height # Flip Y for image coords
        
        # Bounds Check
        ix = int(screen_x)
        iy = int(screen_y)
        
        if 0 <= ix < width and 0 <= iy < height:
            # Splat density
            # Depth weighting could be added here (1/c_w), but flat 1.0 looks good for "glowing" gas
            cuda.atomic.add(grid, (iy, ix), 1.0)

# Allocate Density Buffer
density_grid_d = cp.zeros((RES, RES), dtype=cp.float32)

In [ ]:
def generate_3d_frame(pos_d, step):
    """
    Orchestrates the 3D frame generation.
    1. Computes Host MVP matrix.
    2. Sends MVP to Device.
    3. Runs Kernel.
    4. Colors and converts to Image.
    """
    # 1. Clear Grid
    density_grid_d.fill(0)
    
    # 2. Compute Matrix (Host)
    box_center = (BOX_SIZE/2, BOX_SIZE/2, BOX_SIZE/2)
    mvp_host = get_mvp_matrix(step, box_center, BOX_SIZE)
    mvp_device = cp.asarray(mvp_host) # Transfer to GPU
    
    # 3. Render
    render_3d_density_kernel[BPG, TPB](pos_d, density_grid_d, mvp_device, RES, RES)
    
    # 4. Coloring (Logarithmic Transfer Function)
    img_d = cp.log1p(density_grid_d)
    v_max = cp.max(img_d) + 1e-5
    img_d /= v_max
    
    # "Cosmic Web" Palette
    r = cp.clip(img_d * 1.5 - 0.5, 0, 1) 
    g = cp.clip(img_d * 0.8, 0, 1)
    b = cp.clip(img_d * 0.8 + 0.2, 0, 1)
    
    mask = (img_d > 0.0)
    r *= mask
    g *= mask
    b *= mask
    
    rgb_d = cp.stack((r, g, b), axis=-1)
    
    return Image.fromarray((rgb_d.get() * 255).astype(np.uint8))

In [ ]:
def generate_3d_frame_gpu(pos_d, step, video_buffer_d):
    """
    Renders the frame and stores it directly into the GPU video buffer.
    No CPU transfer happens here.
    """
    density_grid_d.fill(0)
    
    # 2. Compute Matrix (Host side - very fast)
    # We calculate camera position based on 'step'
    box_center = (BOX_SIZE/2, BOX_SIZE/2, BOX_SIZE/2)
    mvp_host = get_mvp_matrix(step, box_center, BOX_SIZE)
    mvp_device = cp.asarray(mvp_host) 
    
    # 3. Render Geometry
    render_3d_density_kernel[BPG, TPB](pos_d, density_grid_d, mvp_device, RES, RES)
    
    # 4. Coloring (All on GPU)
    img_d = cp.log1p(density_grid_d)
    v_max = cp.max(img_d) + 1e-5
    img_d /= v_max
    
    # "Cosmic Web" Palette
    r = cp.clip(img_d * 1.5 - 0.5, 0, 1) 
    g = cp.clip(img_d * 0.8, 0, 1)
    b = cp.clip(img_d * 0.8 + 0.2, 0, 1)
    
    mask = (img_d > 0.0)
    r *= mask
    g *= mask
    b *= mask
    
    # Stack channels
    rgb_d = cp.stack((r, g, b), axis=-1)
    
    # 5. Store in the Main Video Buffer (Cast to uint8 to save VRAM)
    # video_buffer_d is shape (N_STEPS, RES, RES, 3)
    video_buffer_d[step] = (rgb_d * 255).astype(cp.uint8)

In [ ]:
def compile_video(image_folder, output_file, fps=30):
    """
    Compiles a sequence of PNGs into an MP4 using FFmpeg.
    """
    print("Compiling Video...")
    
    # FFmpeg command:
    # -y: Overwrite output
    # -framerate: Input FPS
    # -i: Input pattern
    # -c:v libx264: H.264 Codec
    # -pix_fmt yuv420p: Ensure compatibility with players
    cmd = [
        'ffmpeg',
        '-y',
        '-framerate', str(fps),
        '-i', f'{image_folder}/frame_%04d.png',
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        output_file
    ]
    
    try:
        subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Video saved successfully: {output_file}")
    except subprocess.CalledProcessError as e:
        print("Error compiling video.")
        print(e.stderr.decode())

### Phase 5 : Loop and Simulation

>The cycle here is the following: execute the full Barnes-Hut cycle (Sort $\to$ Build $\to$ Mass $\to$ Force $\to$ Integrate)


>All memory allocation is done at once:

In [ ]:
# --- MEMORY ALLOCATION ---
children_d = cp.full((num_nodes, 2), -1, dtype=cp.int32)
parents_d = cp.full(num_nodes, -1, dtype=cp.int32)
node_mass_d = cp.zeros(num_nodes, dtype=cp.float32)
node_com_d = cp.zeros((num_nodes, 3), dtype=cp.float32)
counters_d = cp.zeros(num_nodes, dtype=cp.int32)
force_d = cp.zeros((N_PARTICLES, 3), dtype=cp.float32)
root_idx_d = cp.zeros(1, dtype=cp.int32)

node_min_d = cp.zeros((num_nodes, 3), dtype=cp.float32) #Bottleneck compute forces
node_max_d = cp.zeros((num_nodes, 3), dtype=cp.float32)

>Now let's compute initial disturbation with Zel'dovich algorithm

In [ ]:
# --- INITIALIZATION ---
print(f"Initializing Simulation: {N_PARTICLES} particles")
pos_d, vel_d, mass_d = generate_zeldovich_ics(N_GRID, BOX_SIZE, spectral_index=2.2, amplitude=25.0)
a = A_START
delta_a = (A_END - A_START) / N_STEPS

print(f"Allocating VRAM for {N_STEPS} frames... ({N_STEPS * RES**2 * 3 / 1e9:.2f} GB)")
video_buffer_d = cp.zeros((N_STEPS, RES, RES, 3), dtype=cp.uint8)

>And finally, maybe the center of the programm, looping everything and storing frames.

In [ ]:

print(f"Starting Render -> {OUTPUT_DIR}/")


t_start = time.time()
cuda.synchronize()

# Create the progress bar object
with tqdm(range(N_STEPS), desc="Simulating") as pbar:
    
    for step in pbar:
        
        H_current = H0 * (a ** -1.5)
        if H_current > MAX_H_RATE: H_current = MAX_H_RATE
        drag_factor = 1.0 / (1.0 + 2.0 * H_current * DT) 

        pos_d, vel_d, mass_d, codes_d = sort_bodies(pos_d, vel_d, mass_d)
        
        children_d.fill(-1)
        parents_d.fill(-1)
        build_radix_tree_kernel[BPG, TPB](codes_d, children_d, parents_d)


        counters_d.fill(0) #reset counters
        node_mass_d.fill(0)
        node_com_d.fill(0)
        node_min_d.fill(0) 
        node_max_d.fill(0)

        compute_multipoles_kernel[BPG, TPB](pos_d, mass_d, children_d, parents_d, 
                                            node_mass_d, node_com_d, 
                                            node_min_d, node_max_d, counters_d)
        find_root_kernel[BPG, TPB](parents_d, root_idx_d)
        
        for i_offset in range(0, N_PARTICLES, BATCH_SIZE): #Batching logic to avoid TDR
            current_batch = min(BATCH_SIZE, N_PARTICLES - i_offset)
            blocks = (current_batch + TPB - 1) // TPB
            compute_forces_kernel[blocks, TPB](pos_d, mass_d, children_d, 
                                               node_mass_d, node_com_d, 
                                               node_min_d, node_max_d, 
                                               force_d, THETA, G, SOFTENING, BOX_SIZE, 
                                               root_idx_d, i_offset)
            
        integrate_kernel[BPG, TPB](pos_d, vel_d, force_d, DT, BOX_SIZE, drag_factor)
    
        generate_3d_frame_gpu(pos_d, step, video_buffer_d)
        a += delta_a

cuda.synchronize()

>Now that the simulation is done and that all frames are stored in the VRAM, everything can be sent back to CPU to be compiled into a nice video.

In [ ]:
def save_frame_to_disk(idx, frame_data, output_dir):
    """
    Helper function for the thread pool
    """
    img = Image.fromarray(frame_data)
    img.save(f"{output_dir}/frame_{idx:04d}.png")

print("Transferring video from GPU to CPU...")
video_buffer_host = video_buffer_d.get() # The big transfer (1.8GB)
print("Transfer Complete. Saving frames to disk...")

# Parallel saving (Uses CPU cores efficiently)
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(N_STEPS):
        futures.append(executor.submit(save_frame_to_disk, i, video_buffer_host[i], OUTPUT_DIR))
    
    # Wait for all to finish
    for _ in tqdm(concurrent.futures.as_completed(futures), total=N_STEPS, desc="Writing PNGs"):
        pass

print("All frames saved.")

# Compile Video
compile_video(OUTPUT_DIR, VIDEO_NAME, fps=60)

>Up to this point the pattern was the following Render $\to$ Download $\to$ Save $\to$ Repeat which was not optimal. To tackle that performance issue, everything is now stored in device memory and all frames are sent back at the end of the simulation for compilation.

>The following cell is for profiling.

In [ ]:

start_event = cp.cuda.Event()
end_event = cp.cuda.Event()

def profile_step():
    #Sort
    start_event.record()
    pos_s, vel_s, mass_s, codes_s = sort_bodies(pos_d, vel_d, mass_d)
    end_event.record()
    end_event.synchronize()
    t_sort = cp.cuda.get_elapsed_time(start_event, end_event)

    #Build Tree
    start_event.record()
    build_radix_tree_kernel[BPG, TPB](codes_s, children_d, parents_d)
    end_event.record()
    end_event.synchronize()
    t_build = cp.cuda.get_elapsed_time(start_event, end_event)

    #Multipoles
    start_event.record()
    counters_d.fill(0)
    node_mass_d.fill(0)
    node_com_d.fill(0)
    compute_multipoles_kernel[BPG, TPB](pos_s, mass_s, children_d, parents_d, 
                                        node_mass_d, node_com_d, 
                                        node_min_d, node_max_d, counters_d)
    find_root_kernel[BPG, TPB](parents_d, root_idx_d)
    end_event.record()
    end_event.synchronize()
    t_mass = cp.cuda.get_elapsed_time(start_event, end_event)

    #Forces
    start_event.record()
    compute_forces_kernel[BPG, TPB](pos_s, mass_s, children_d, node_mass_d, node_com_d, 
                                    node_min_d, node_max_d, force_d, 
                                    THETA, G, SOFTENING, BOX_SIZE, root_idx_d, 0)
    end_event.record()
    end_event.synchronize()
    t_force = cp.cuda.get_elapsed_time(start_event, end_event)

    print(f"Sort: {t_sort:.2f}ms")
    print(f"Tree Build: {t_build:.2f}ms")
    print(f"Mass Calc: {t_mass:.2f}ms")
    print(f"Force Calc: {t_force:.2f}ms")

print("Profiling one step...")
profile_step()

![image.png](resources/image.png)

>The programm is running at ~7s/it which is good but not great. Some profiling gives us the culprit:\
`Profiling one step...`\
`Sort: 25.41ms`\
`Tree Build: 2.27ms`\
`Mass Calc: 15.90ms`\
`Force Calc: 6845.38ms`

>The force calculation is taking more than 6s to compute! And this is likely due to the fact that the previous implementation was forcing the GPU to "open" any nodes closer than $10\%$ of the box size. Making the complexity in dense areas drop to O($N^2$).\
>Fixing this requires to add [Axis-Aligned Bounding Box logic](https://en.wikipedia.org/wiki/Bounding_volume) to the multipole compute kernel.\
>Adding to that a batching logic to help with smooth computation (No spikes).





>Once implemented the profiling gives us ~2.8 s/it and the GPU is now running at ~98% without spikes.

### Phase 6: Results

>Some Renders:\
>The folowing ones were early results with basic random initial condition and/or basic euler integrations.

| "The messy magma"  | "Increased energy" |
| ------------- |:-------------:|
| ![old_render4](resources/render_4.webp "Old render 1")      | ![old_render5](resources/render_5.webp "Old render 1")     |
| "The Wrong Zel'Dovich"      | "The chill guy"     |
| ![old_render6](resources/render_6.webp "Old render 1")      | ![old_render7](resources/render_7.webp "Old render 1")     |
| "The Pink supernova"      | "The Blue Supernova"     |
| ![old_render1](resources/1_1_test_animation.webp "Old render 1")      | ![old_render1](resources/1_2_test_render.webp "Old render 1")     |


>This one had a power function of $k^{-3}$ which puts almost all the energy into large-scale waves (low $k$) and very little into small-scale waves (high $k$).
>The hubble drag and softening were too strong.\
>This said some galaxies can be seen!
![old_render1](resources/render_3.webp "Old render 1")